### 청깅 중 의미를 고려한 청킹
1. 문장 길이 단위로 하면 내용의 완결성이 떨어져서 검색의 정확도를 떨어뜨린다.
2. 이것은 각 문장을 임베딩하고 각 문장끼리 유사도를 고려해서 유사도가 높은 것끼지 청크되어 분할 됨.

In [2]:
import os
import urllib.request
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

In [3]:
from dotenv import load_dotenv
load_dotenv()
openAPI_KEY = os.getenv('openAPI')

os.environ['OPENAI_API_KEY'] =openAPI_KEY

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/chatgpt-kr/openai-api-tutorial/main/ch06/test.txt", filename="test.txt")

('test.txt', <http.client.HTTPMessage at 0x1a65ae39a90>)

### 데이터 다운로드

In [4]:
with open("test.txt", encoding="utf-8") as f:
    file = f.read()

print("텍스트 길이:", len(file))

텍스트 길이: 7460


### SementicChunker 사용하기
1. 백분위수(디폴트)
2. 표준편차
3. 사분위수

이렇게 3가지 방식으로 하되 모두 유사도(코사인 거리=1-유사도)를 이용한다.

In [5]:
# 내부적으로 임베딩을 통하여 유사도 높은 문장끼리 청크
text_splitter = SemanticChunker(OpenAIEmbeddings())
texts = text_splitter.create_documents([file])
print("분활된 청크 수:", len(texts))

분활된 청크 수: 10


In [15]:
# 아래 내용을 보면 서로 의미 있게 청크되어 있다.
print(texts[4])
print()
print(texts[5])
print()
print(texts[6])

page_content='그러나 현실은 냉혹했다. 연이은 탈락 통지에 그는 좌절감을 느꼈다. "내가 부족한 걸까?" 자신을 의심하기 시작했다. 결국 그는 작은 스타트업에서 일하기 시작했다. 급여는 많지 않았지만, 다양한 경험을 쌓을 수 있었다. 그는 이 시기를 자신의 실력을 갈고닦는 기회로 삼았다. 밤을 새워가며 새로운 기술을 익혔고, 회사 프로젝트에 혁신적인 아이디어를 제안했다. 2년이 지나자 그의 노력이 빛을 발하기 시작했다. 그가 개발한 알고리즘이 업계의 주목을 받게 된 것이다. 여러 기업에서 그에게 이직 제안을 해왔고, 그는 더 큰 회사로 옮길 수 있었다. 그러나 여전히 그의 꿈은 더 컸다. 그는 세계적인 IT 기업들이 모여 있는 미국 실리콘밸리로의 진출을 꿈꿨다. 수많은 지원서를 보냈지만, 대부분 거절당했다.'

page_content='언어 장벽과 경력 부족이 주된 이유였다. 그러나 박민호는 포기하지 않았다. 퇴근 후에는 영어 학원에 다녔고, 주말에는 국제 컨퍼런스에 참가해 네트워크를 넓혔다. 이러한 노력 끝에 2010년, 그는 마침내 미국 실리콘밸리의 한 중소 IT 기업에 취직하게 되었다. 실리콘밸리에서의 첫 직장은 그에게 새로운 도전과 기회를 제공했다. 그는 자신의 꿈을 이루기 위한 첫 발걸음을 내딛었다는 생각에 가슴이 뛰었다. 해외에서의 도전과 성공
미국에서의 생활은 결코 쉬운 일이 아니었다. 문화적 차이와 언어 장벽, 그리고 새로운 환경에 적응해야 했다. 처음 몇 달 동안 박민호는 매일 밤 한국으로 돌아가고 싶다는 생각을 했다. 회의 중 동료들의 농담을 이해하지 못해 웃지 못할 때도 많았고, 업무 지시를 제대로 이해하지 못해 실수를 저지르기도 했다. 그러나 박민호는 포기하지 않았다. 그는 퇴근 후에도 영어 공부를 계속했고, 주말에는 현지 문화를 체험하기 위해 다양한 활동에 참여했다. 동료들과의 대화에서 놓친 부분이 있으면 나중에 따로 물어보며 이해하려 노력했다. 이러한 그의 성실함과 열정은 동료들에게 좋은 인상을 주었다. 점차 시간이 지나면서

#### 먼저 백분위수로 문장을 나누기
1. 결국 상위 백분위라는 것은 연속된 두 문장간 의미가 없다(잘 생각해야 함)
2. 그래서 의미가 없으니 거기서 분리 하는 것이다.
3. 그러면 의문이 그냥 유사도거리(1-유사도) 하지 말고 그냥 유사도 해서 유사도가 적은 것 해도 되지 않나

나머지 표준편차, 사분위수는 교체 참고

In [16]:
text_splitter = SemanticChunker(
    OpenAIEmbeddings(),
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=95 # 교재에 아주 잘 나와 있음
)

texts = text_splitter.create_documents([file])
print("분활된 청크 수:", len(texts))

분활된 청크 수: 10
